In [1]:
%load_ext autotime

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from keras.applications.densenet import DenseNet121
from keras.utils import np_utils
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import optimizers
from keras import metrics
from keras import models

from keras.utils import plot_model, multi_gpu_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


from pathlib import Path
import pickle

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

import utils
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
classifier = '3_1_body_part_classifier_40'
base_model = '2_1_submodel_335'

time: 508 µs


In [3]:
model_name = '3_2_ensemble'

time: 5.97 ms


In [ ]:
with tf.device('/CPU:0'):
    classifier = models.load_model(f'trained_models/{classifier}.h5')
    classifier = models.Model(input=classifier.layers[-2].inputs[0], output=classifier.layers[-2].outputs[0])
    for mid_layer in classifier.layers:
        mid_layer.trainable = False

    sub_models = []
    for _ in range(7):
        dn121 = DenseNet121(input_tensor=classifier.input, classes=1, weights=None, include_top=False)
        model = models.Model(input=dn121.input, output=layers.Dense(1, activation='sigmoid')(layers.Flatten()(dn121.output)))
        model.load_weights(f'trained_models/{base_model}.h5')
        sub_models.append(model)
        del dn121
        del model
        
    i = 1
    for model in sub_models:
        for mid_layer in model.layers:
            mid_layer.name = str(i) + '_' + mid_layer.name
        i += 1
    for mid_layer in classifier.layers:
        mid_layer.name = 'c_' + mid_layer.name
        
    concat = layers.Concatenate(1)([model.output for model in sub_models])
    dot = layers.Dot(axes=1)([concat, classifier.output])
    
    ensemble = models.Model(input=classifier.input, output=dot)

In [6]:
plot_model(ensemble, to_file='ensemble.png')

time: 10 s


In [7]:
model = multi_gpu_model(ensemble, gpus=2)

time: 10min 23s


# Prep Data

In [12]:
x_train, y_train, x_test, y_test = utils.read_mura_pickle()

INFO:utils.utils:loading data/MURA-v1.1/x_train.pkl
INFO:utils.utils:loading data/MURA-v1.1/y_train.pkl
INFO:utils.utils:loading data/MURA-v1.1/x_valid.pkl
INFO:utils.utils:loading data/MURA-v1.1/y_valid.pkl


time: 2.35 s


In [13]:
size = x_train.shape[1]

time: 680 µs


In [14]:
x_train = utils.normalize_pixels(x_train)
x_test = utils.normalize_pixels(x_test)
x_train = x_train.reshape(x_train.shape[0], size, size, 1)
x_test = x_test.reshape(x_test.shape[0], size, size, 1)

time: 3.98 s


In [15]:
x_train.shape

(36808, 256, 256, 1)

time: 2.76 ms


In [16]:
datagen = ImageDataGenerator(
    rotation_range=360,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=[1, 1.2],
    fill_mode = 'constant',
    cval=0,
    horizontal_flip=True,
    vertical_flip=True)

time: 4.28 ms


In [17]:
datagen.fit(x_train)

time: 4.17 s


# Training

In [22]:
starting_epoch = 0
train_round = 0
batch_size = 8
epochs_per_eval = 10
epochs_per_save = 30
total_epochs = 210
train_history = {'loss': [], 'binary_accuracy': []}
val_per_image_history = {'accuracy': [], 'kappa': [], 'precision': [], 'recall': []}
val_per_study_history = {'accuracy': [], 'kappa': [], 'precision': [], 'recall': []}

time: 3.68 ms


In [ ]:
# with tf.device('/CPU:0'):
#     ensemble = models.load_model(f'./trained_models/{model_name}_235.h5')
#     model = multi_gpu_model(ensemble, gpus=2)

In [ ]:
# with open(f'./trained_models/{model_name}_train_history.pkl', 'rb') as pkl_file:
#     train_history = pickle.load(pkl_file)
# with open(f'./trained_models/{model_name}_val_per_image_history.pkl', 'rb') as pkl_file:
#     val_per_image_history = pickle.load(pkl_file)
# with open(f'./trained_models/{model_name}_val_per_study_history.pkl', 'rb') as pkl_file:
#     val_per_study_history = pickle.load(pkl_file)

In [ ]:
while (train_round+1) * epochs_per_eval + starting_epoch <= total_epochs:
    print(f'executing epochs {train_round * epochs_per_eval + 1 + starting_epoch}-{(train_round+1) * epochs_per_eval + starting_epoch}')
    
    if train_round * epochs_per_eval + 1 + starting_epoch <= 200:
        opt = optimizers.Adam()
    else:
        lr = 0.001 * (0.9 ** train_round)
        opt = optimizers.sgd(lr=lr, momentum=0.9, nesterov=True)
        
    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics=[metrics.binary_accuracy])
    
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                           steps_per_epoch = len(x_train) / batch_size, epochs=epochs_per_eval, verbose=2)
    
    train_history['loss'].extend(model.history.history['loss'])
    train_history['binary_accuracy'].extend(model.history.history['binary_accuracy'])
    
    y_valid_hat = model.predict(x_test)
    true_label = np.round(y_test)
    pred_label = np.round(y_valid_hat)
    evaluate = utils.MURAMetrics(true_label, pred_label)
    
    per_image_metrics = evaluate.report_by_image()
    for key in per_image_metrics:
        val_per_image_history[key].append(per_image_metrics[key])
        print(f'Valid per image {key}: {per_image_metrics[key]}')
    
    per_study_metrics = evaluate.report_by_study()
    for key in per_image_metrics:
        val_per_study_history[key].append(per_study_metrics[key])
        print(f'Valid per study {key}: {per_study_metrics[key]}')
    
    cur_epoch = (train_round+1) * epochs_per_eval
    if cur_epoch % epochs_per_save == 0:
        print(f'Saving Model to trained_models/{model_name}_{cur_epoch + starting_epoch}.h5...')
        model.save(f'./trained_models/{model_name}_{cur_epoch + starting_epoch}.h5')
    
    print('Saving evaluation metrics history...')
    with open(f'./trained_models/{model_name}_train_history.pkl', 'wb') as pkl_file:
        pickle.dump(train_history, pkl_file)
    with open(f'./trained_models/{model_name}_val_per_image_history.pkl', 'wb') as pkl_file:
        pickle.dump(val_per_image_history, pkl_file)
    with open(f'./trained_models/{model_name}_val_per_study_history.pkl', 'wb') as pkl_file:
        pickle.dump(val_per_study_history, pkl_file)
    
    train_round += 1

executing epochs 1-10
Epoch 1/10
